In [18]:
import torch

import transformers
from transformers import pipeline

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification

In [2]:
# run a model end to end, using defaults

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
        "top 10 electric bikes of 2022",
        "This is insanely cool",
        "What a shot!",
        "avoid these beauty products"
    ]
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455},
 {'label': 'POSITIVE', 'score': 0.9988521337509155},
 {'label': 'POSITIVE', 'score': 0.9998574256896973},
 {'label': 'POSITIVE', 'score': 0.9977614879608154},
 {'label': 'NEGATIVE', 'score': 0.9993860721588135}]

In [6]:
# run the tokenizer part

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
raw_inputs = [
    "top 10 electric bikes of 2022",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  2327,  2184,  3751, 18105,  1997, 16798,  2475,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0]])}


In [10]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)


torch.Size([2, 9, 768])


In [13]:
type(inputs)

transformers.tokenization_utils_base.BatchEncoding

In [14]:
type(outputs)

transformers.modeling_outputs.BaseModelOutput

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [17]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-3.3423,  3.4264],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [19]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[1.1479e-03, 9.9885e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)
